# Data Preprocessing

This notebook will be a part of the **Data Preprocessing/Cleaning** node of the Klee Visual Analytics project.

Data cleaning is the process of preparing data for analysis by removing or modifying data that is incorrect, incomplete, irrelevant, duplicated, or improperly formatted. Usually *Raw data* can come in all kinds of strange distribution and non uniform formats, that it makes analyzing of data and creation of model very difficult. In this notebook we will be using multiple libraries for performing common data cleaning steps on tabular dataset.


#### Usage: 
This notebook can perform some data cleaning operations on any type of tabular dataset and read it in format of CSV or Datasets. Refer to **Load the dataset** part of the notebook to play around your custom dataset file

#### Methods used in this notebook:
- Removing unwanted features
- Imputing missing values
- Scaling
- Normalization
- Encoding for categorical columns

### Libraries used 
- We use ``ScikitLearn`` to investigate and clean the dataset. Cleaning the input data is a very important step before moving to modeling part of the data Science pipeline, as it can potentially harm model's prediction. 

#### Input: 
The input to this notebook is Tabular dataset.

#### Output:
Output of this notebook is a clean form of the same dataset.

![Klee - Visual Analytics](https://github.com/nikbearbrown/Visual_Analytics/blob/main/IMG/Klee_Visual_Analytics.png?raw=true)


YouTube - https://www.youtube.com/c/NikBearBrown    
GitHub - https://github.com/nikbearbrown/Visual_Analytics   
Kaggle - https://www.kaggle.com/nikbearbrown   
Klee.ai (Visual AI) - http://klee.ai    



In [2]:
# Imports

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer

In [3]:
# loading dataset
url = "https://github.com/nikbearbrown/Visual_Analytics/raw/main/CSV/titanic_dataset.csv"
df=pd.read_csv(url)
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


### Excercise

Do these data types make sense? If not convert them.
Should Pclass be categorical or numeric?


#### Removing Unwanted Features
Those features which contain constant values (i.e. only one value for all the outputs or target values) in the dataset are known as Constant Features. These features don’t provide any information to the target feature. These are redundant data available in the dataset. Presence of this feature has no effect on the target, so it is good to remove these features from the dataset.

In [7]:
# removing unwanted features

def remove_features(dataframe,feature_name_list):
    """
    Deleting a features from dataframe
    """
    for feature_name in feature_name_list:
        try:
            del dataframe[feature_name]
        except:
            print("Can't delete feature %s" % (feature_name))          

In [8]:
# removing the feature from the dataset
remove_features(df,['PassengerId', 'Name'])
df.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,3,female,47.0,1,0,363272,7.0000,NaN,S
2,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,3,male,27.0,0,0,315154,8.6625,NaN,S
4,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
# Identify object columns
print(df.select_dtypes(['object']).columns)

# Identify integer columns
print(df.select_dtypes(['int64']).columns)

# Identify numeric columns
print(df.select_dtypes(include=np.number).columns.tolist())

Index([], dtype='object')
Index(['Pclass'], dtype='object')
['Pclass', 'Age', 'Fare']


In [10]:
# What does this tell you? Why isn't it df.shape()?
print(df.shape)

# What are the variable names?
X=df.columns.values.tolist()
print(X)

# How many variables?
print(len(X))

(418, 9)
['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
9


In [12]:
# Convert numeric to categorical variables
actually_categorical = ['Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked']                      
for col in actually_categorical:
    df[col] = df[col].astype('category')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Pclass    418 non-null    int64   
 1   Sex       418 non-null    category
 2   Age       332 non-null    float64 
 3   SibSp     418 non-null    category
 4   Parch     418 non-null    category
 5   Ticket    418 non-null    category
 6   Fare      417 non-null    float64 
 7   Cabin     91 non-null     category
 8   Embarked  418 non-null    category
dtypes: category(6), float64(2), int64(1)
memory usage: 29.6 KB


### Impute misssing values
Real world datasets usually contain alot of missing values, often encoded as blanks, NaNs or other placeholders. Such datasets however are incompatible with alot of functions and models which assume that all values in an array are numerical, and that all have and hold meaning. A basic strategy to use incomplete datasets is to discard entire rows and/or columns containing missing values or to impute the missing values, i.e., to infer them from the known part of the data. 
- **categorical value** represented as string values or pandas categoricals when using the 'most_frequent' or 'constant' strategy
- **numerical values** are inputed using "mean" of the feature
\
\
> Methods to be included:
- Nearest neighbour imputation
- A model trained on exisiting data to impute missing values

In [35]:
def impute_missing_values(inputData, columns = list()):

    ContinuousColumns = inputData.select_dtypes(include=np.number).columns.tolist()
    CategoricalColumns = inputData.select_dtypes(['category']).columns
    print ("Continuous ", ContinuousColumns)
    print ("Categorical ", CategoricalColumns)
    
    
    if columns:
        CategoricalColumns = list(set.intersection(set(columns), set(CategoricalColumns)))
        print(CategoricalColumns)
        ContinuousColumns = list(set.intersection(set(columns), set(ContinuousColumns)))
        print(ContinuousColumns)   
        
        
    # for all continuous columns
    if ContinuousColumns:    
        imp = SimpleImputer()
        imp.fit(inputData[ContinuousColumns])
        inputData[ContinuousColumns] = imp.transform(inputData[ContinuousColumns])
        
    # # for all categorical columns
    if CategoricalColumns:    
        imp = SimpleImputer(strategy="most_frequent")
        imp.fit(inputData[CategoricalColumns])
        inputData[CategoricalColumns] = imp.transform(inputData[CategoricalColumns])
        
    return inputData             
        
                
        

In [37]:
df.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        1
Cabin       0
Embarked    0
dtype: int64

In [40]:
impute_missing_values(df,['Cabin','Age', 'Fare'])

Continuous  ['Pclass', 'Age', 'Fare']
Categorical  Index(['Sex', 'SibSp', 'Parch', 'Ticket', 'Embarked'], dtype='object')
[]
['Fare', 'Age']


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,34.50000,0,0,330911,7.8292,B57 B59 B63 B66,Q
1,3,female,47.00000,1,0,363272,7.0000,B57 B59 B63 B66,S
2,2,male,62.00000,0,0,240276,9.6875,B57 B59 B63 B66,Q
3,3,male,27.00000,0,0,315154,8.6625,B57 B59 B63 B66,S
4,3,female,22.00000,1,1,3101298,12.2875,B57 B59 B63 B66,S
...,...,...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S
414,1,female,39.00000,0,0,PC 17758,108.9000,C105,C
415,3,male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S
416,3,male,30.27259,0,0,359309,8.0500,B57 B59 B63 B66,S


In [41]:
df.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

### Scaling Input
Input variables may have different units (e.g. feet, kilometers, and hours) that, in turn, may mean the variables have different scales. The two most popular techniques for scaling numerical data prior to modeling are **normalization** and **standardization**. Normalization scales each input variable separately to the range 0-1, which is the range for floating-point values where we have the most precision. Standardization scales each input variable separately by subtracting the mean (called centering) and dividing by the standard deviation to shift the distribution to have a mean of zero and a standard deviation of one.

In [42]:
def scaling_input(inputData):
    """
    Changing the range of numerical input features of dataset

    """
    columnNames = inputData.select_dtypes(include='number').columns
    inputData[columnNames] = preprocessing.scale(inputData[columnNames])
    
    return inputData

In [43]:
scaling_input(df)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0.873482,male,0.334993,0,0,330911,-0.498407,B57 B59 B63 B66,Q
1,0.873482,female,1.325530,1,0,363272,-0.513274,B57 B59 B63 B66,S
2,-0.315819,male,2.514175,0,0,240276,-0.465088,B57 B59 B63 B66,Q
3,0.873482,male,-0.259330,0,0,315154,-0.483466,B57 B59 B63 B66,S
4,0.873482,female,-0.655545,1,1,3101298,-0.418471,B57 B59 B63 B66,S
...,...,...,...,...,...,...,...,...,...
413,0.873482,male,0.000000,0,0,A.5. 3236,-0.494448,B57 B59 B63 B66,S
414,-1.505120,female,0.691586,0,0,PC 17758,1.313753,C105,C
415,0.873482,male,0.651965,0,0,SOTON/O.Q. 3101262,-0.508792,B57 B59 B63 B66,S
416,0.873482,male,0.000000,0,0,359309,-0.494448,B57 B59 B63 B66,S


In [44]:
def normalize_data(inputData, columnNames):
    """
    changing the shape of distribution of data
    """

    if not columnNames:
        columnNames = inputData.select_dtypes(include='number').columns

    transformer = preprocessing.Normalizer().fit(inputData[columnNames])
    inputData[columnNames] = transformer.transform(inputData[columnNames])
    
    return inputData

In [45]:
normalize_data(df,["Pclass"])

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,male,0.334993,0,0,330911,-0.498407,B57 B59 B63 B66,Q
1,1.0,female,1.325530,1,0,363272,-0.513274,B57 B59 B63 B66,S
2,-1.0,male,2.514175,0,0,240276,-0.465088,B57 B59 B63 B66,Q
3,1.0,male,-0.259330,0,0,315154,-0.483466,B57 B59 B63 B66,S
4,1.0,female,-0.655545,1,1,3101298,-0.418471,B57 B59 B63 B66,S
...,...,...,...,...,...,...,...,...,...
413,1.0,male,0.000000,0,0,A.5. 3236,-0.494448,B57 B59 B63 B66,S
414,-1.0,female,0.691586,0,0,PC 17758,1.313753,C105,C
415,1.0,male,0.651965,0,0,SOTON/O.Q. 3101262,-0.508792,B57 B59 B63 B66,S
416,1.0,male,0.000000,0,0,359309,-0.494448,B57 B59 B63 B66,S


### One Hot Encoding
A one hot encoding is a representation of categorical variables as binary vectors.This first requires that the categorical values be mapped to integer values. Then, each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1.

In [46]:
            
def one_hot_encode(inputData):
    """
    Encoding of Categorical features with one hot numeric array
    """

    columnNames= inputData.select_dtypes(include=['object']).columns 
    print(columnNames)
    print(inputData[columnNames])
    drop_enc = preprocessing.OneHotEncoder(drop='first')
    df = pd.DataFrame(drop_enc.fit_transform(inputData[columnNames]).toarray())
    inputData = inputData.join(df)
    
    return inputData

In [47]:
one_hot_encode(df)

Index(['Cabin'], dtype='object')
               Cabin
0    B57 B59 B63 B66
1    B57 B59 B63 B66
2    B57 B59 B63 B66
3    B57 B59 B63 B66
4    B57 B59 B63 B66
..               ...
413  B57 B59 B63 B66
414             C105
415  B57 B59 B63 B66
416  B57 B59 B63 B66
417  B57 B59 B63 B66

[418 rows x 1 columns]


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,0,...,65,66,67,68,69,70,71,72,73,74
0,1.0,male,0.334993,0,0,330911,-0.498407,B57 B59 B63 B66,Q,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,female,1.325530,1,0,363272,-0.513274,B57 B59 B63 B66,S,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.0,male,2.514175,0,0,240276,-0.465088,B57 B59 B63 B66,Q,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,male,-0.259330,0,0,315154,-0.483466,B57 B59 B63 B66,S,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,female,-0.655545,1,1,3101298,-0.418471,B57 B59 B63 B66,S,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1.0,male,0.000000,0,0,A.5. 3236,-0.494448,B57 B59 B63 B66,S,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
414,-1.0,female,0.691586,0,0,PC 17758,1.313753,C105,C,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415,1.0,male,0.651965,0,0,SOTON/O.Q. 3101262,-0.508792,B57 B59 B63 B66,S,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
416,1.0,male,0.000000,0,0,359309,-0.494448,B57 B59 B63 B66,S,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
